# SVM i XGBoost - Zadania praktyczne z danymi Attrition

## Cel ćwiczeń
W tych zadaniach będziemy pracować z rzeczywistym zbiorem danych HR dotyczącym odejść pracowników (attrition). Nauczimy się:
- Przeprowadzać eksploracyjną analizę danych (EDA)
- Przygotowywać dane do modelowania
- Implementować i porównywać modele SVM i XGBoost
- Optymalizować hiperparametry
- Interpretować wyniki i wyciągać wnioski biznesowe

## Informacje o zbiorze danych
- **Liczba rekordów**: 59,598 pracowników
- **Liczba cech**: 24 (8 numerycznych, 16 kategorycznych)
- **Target**: Attrition (Stayed/Left)
- **Braki danych**: Brak
- **Balans klas**: Stayed ≈ 52.5%, Left ≈ 47.5%

In [ ]:
# Importy bibliotek - uzupełnij brakujące
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Ustawienia wykresów
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

# Zadanie 1: Wczytanie i podstawowe poznanie danych 📊

## Cel
Zapoznać się ze strukturą i jakością danych przed przystąpieniem do modelowania.

## Do zrobienia:

### 1.1 Wczytanie danych
- Wczytaj plik `attrition.csv` do DataFrame
- Wyświetl podstawowe informacje o zbiorze: `info()`, `describe()`, `shape`
- Sprawdź czy są braki danych

In [ ]:
# Twój kod tutaj:
# Wskazówki: pd.read_csv(), .info(), .describe(), .shape, .isnull().sum()

### 1.2 Analiza zmiennej target
- Sprawdź rozkład zmiennej `Attrition` (ile osób zostało/odeszło)
- Oblicz procenty dla każdej kategorii
- Stwórz wykres słupkowy pokazujący rozkład

In [ ]:
# Twój kod tutaj:
# Wskazówki: .value_counts(), .value_counts(normalize=True), sns.countplot()

### 1.3 Identyfikacja typów zmiennych
- Podziel kolumny na numeryczne i kategoryczne
- Wyświetl listę zmiennych numerycznych (powinno być 8)
- Wyświetl listę zmiennych kategorycznych (powinno być 16)
- Sprawdź unikalne wartości w zmiennych kategorycznych

In [ ]:
# Twój kod tutaj:
# Wskazówki: .select_dtypes(), list(), .nunique()
target = 'Attrition'

# Zadanie 2: Eksploracyjna Analiza Danych (EDA) 🔍

## Cel
Zrozumieć wzorce w danych i zidentyfikować potencjalne czynniki wpływające na odejścia.

### 2.1 Analiza zmiennych numerycznych
- Stwórz histogramy dla wszystkich zmiennych numerycznych
- Użyj `subplot` żeby pokazać wszystkie na jednym wykresie
- Sprawdź czy są outliersy używając boxplotów
- Oblicz korelacje między zmiennymi numerycznymi

In [ ]:
# Twój kod tutaj:
# Wskazówki: plt.subplots(), .hist(), sns.boxplot(), .corr(), sns.heatmap()

### 2.2 Analiza zmiennych kategorycznych
- Dla każdej zmiennej kategorycznej sprawdź:
  - Liczbę unikalnych wartości
  - Rozkład częstości każdej kategorii
- Stwórz wykresy słupkowe dla najważniejszych zmiennych kategorycznych
- Zwróć szczególną uwagę na `Job Role`, `Work-Life Balance`, `Job Satisfaction`

In [ ]:
# Twój kod tutaj:
# Wskazówki: for loop, .value_counts(), sns.barplot(), plt.subplots()

### 2.3 Analiza związku zmiennych z target
- Dla zmiennych numerycznych:
  - Stwórz boxploty porównujące rozkłady dla Stayed vs Left
  - Przeprowadź test t-Studenta dla istotnych różnic
- Dla zmiennych kategorycznych:
  - Stwórz tabele krzyżowe (crosstab) z procentami
  - Zidentyfikuj które kategorie mają wyższy % odejść

In [ ]:
# Twój kod tutaj:
# Wskazówki: sns.boxplot(), pd.crosstab(), normalize='index'

### 2.4 Wnioski z EDA
Na podstawie analizy odpowiedz na pytania:
- Które zmienne numeryczne najsilniej różnicują grupy Stayed/Left?
- Które kategorie w zmiennych kategorycznych mają najwyższy % odejść?
- Czy widzisz jakieś interesujące wzorce? (np. overtime vs attrition)
- Które zmienne mogą być najważniejsze dla modelu?

**Twoje obserwacje i wnioski:**
<!-- Zapisz tutaj swoje przemyślenia po EDA -->

# Zadanie 3: Przygotowanie danych do modelowania 🛠️

## Cel
Przekształcić dane surowe w format odpowiedni dla algorytmów ML.

### 3.1 Czyszczenie danych
- Usuń kolumnę `Employee ID` (nie pomaga w predykcji)
- Przekształć zmienną target `Attrition` na wartości binarne (0/1)
- Sprawdź czy wszystkie dane są w odpowiednim formacie

In [ ]:
# Twój kod tutaj:
# Wskazówki: .drop(), .map(), .copy()

### 3.2 Preprocessing Pipeline
- Użyj `ColumnTransformer` do zdefiniowania preprocessingu:
  - `StandardScaler` dla zmiennych numerycznych
  - `OneHotEncoder` dla zmiennych kategorycznych
- Stwórz pipeline łączący preprocessing z modelem
- Podziel dane na X (cechy) i y (target)

In [ ]:
# Twój kod tutaj:
# Wskazówki: ColumnTransformer(), transformers=[...], StandardScaler(), OneHotEncoder()
# Pipeline(), steps=[...]

preprocessor = # ColumnTransformer tutaj

### 3.3 Podział na zbiory train/validation/test
- Podziel dane na train (60%), validation (20%), test (20%)
- Użyj `stratify` żeby zachować proporcje klas
- Ustaw `random_state=42` dla powtarzalności

In [ ]:
# Twój kod tutaj:
# Wskazówki: train_test_split(), stratify=, random_state=42

### 3.4 Uwaga o Preprocessingu
- **Pipeline automatycznie zastosuje preprocessing** podczas .fit() i .predict()
- Nie musisz ręcznie skalować danych - Pipeline to zrobi
- ColumnTransformer zapewni, że odpowiednie transformacje zostaną zastosowane do odpowiednich kolumn

In [ ]:
# Preprocessing jest już zdefiniowany w ColumnTransformer powyżej
# Pipeline automatycznie zastosuje odpowiednie transformacje


# Zadanie 4: Implementacja i trening modelu SVM 🎯

## Cel
Zbudować i zoptymalizować model SVM dla predykcji attrition.

### 4.1 Pipeline SVM
- Stwórz pipeline łączący preprocessor z modelem SVM (kernel='rbf')
- Wytrenuj pipeline na zbiorze treningowym
- Oceń na zbiorze walidacyjnym
- Wyświetl accuracy, classification report i confusion matrix

In [ ]:
# Twój kod tutaj:
# Model bazowy
# svm_base = SVC(kernel='rbf', random_state=42)
# svm_base.fit(X_train_scaled, y_train)

# Predykcje
# y_val_pred = svm_base.predict(X_val_scaled)

# Ocena
# print("Accuracy:", accuracy_score(y_val, y_val_pred))
# print("\nClassification Report:")
# print(classification_report(y_val, y_val_pred))

### 4.2 Testowanie różnych kerneli
- Przetestuj kernele: 'linear', 'rbf', 'poly'
- Dla każdego kernela:
  - Wytrenuj model
  - Oceń na walidacji
  - Porównaj wyniki
- Który kernel działa najlepiej?

In [ ]:
# Twój kod tutaj:
# kernels = ['linear', 'rbf', 'poly']
# results = {}

# for kernel in kernels:
#     svm = SVC(kernel=kernel, random_state=42)
#     svm.fit(X_train_scaled, y_train)
#     y_pred = svm.predict(X_val_scaled)
#     accuracy = accuracy_score(y_val, y_pred)
#     results[kernel] = accuracy
#     print(f"{kernel}: {accuracy:.4f}")

### 4.3 Optymalizacja hiperparametrów
- Użyj `GridSearchCV` do optymalizacji parametrów C i gamma dla najlepszego kernela
- Zakres parametrów:
  - C: [0.1, 1, 10, 100]
  - gamma: ['scale', 'auto', 0.001, 0.01, 0.1, 1]
- Użyj 5-fold cross-validation
- Oceń najlepszy model na zbiorze walidacyjnym

In [ ]:
# Twój kod tutaj:
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]
# }

# grid_search = GridSearchCV(...)
# grid_search.fit(X_train_scaled, y_train)

# print("Najlepsze parametry:", grid_search.best_params_)
# print("Najlepszy wynik CV:", grid_search.best_score_)

### 4.4 Ocena finalnego modelu SVM
- Użyj najlepszego modelu z Grid Search
- Oceń na zbiorze testowym
- Stwórz confusion matrix z ładnymi etykietami
- Oblicz dodatkowe metryki: precision, recall, F1-score

In [ ]:
# Twój kod tutaj:
# best_svm = grid_search.best_estimator_
# y_test_pred = best_svm.predict(X_test_scaled)

# print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
# print("\nClassification Report:")
# print(classification_report(y_test, y_test_pred, target_names=['Stayed', 'Left']))

# Confusion Matrix
# cm = confusion_matrix(y_test, y_test_pred)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#             xticklabels=['Stayed', 'Left'], yticklabels=['Stayed', 'Left'])
# plt.title('SVM - Confusion Matrix')
# plt.show()

# Zadanie 5: Implementacja i trening modelu XGBoost 🚀

## Cel
Zbudować i zoptymalizować model XGBoost i porównać z SVM.

### 5.1 Model bazowy XGBoost
- Stwórz model XGBoost z domyślnymi parametrami
- **Uwaga**: XGBoost nie wymaga standaryzacji, użyj oryginalnych danych
- Wytrenuj i oceń na zbiorze walidacyjnym
- Porównaj wyniki z modelem SVM

In [ ]:
# Twój kod tutaj:
# xgb_base = XGBClassifier(random_state=42, eval_metric='logloss')
# xgb_base.fit(X_train, y_train)

# y_val_pred_xgb = xgb_base.predict(X_val)
# print("XGBoost Accuracy:", accuracy_score(y_val, y_val_pred_xgb))
# print("\nClassification Report:")
# print(classification_report(y_val, y_val_pred_xgb))

### 5.2 Analiza ważności cech
- Wyświetl feature importance z modelu XGBoost
- Stwórz wykres słupkowy 15 najważniejszych cech
- Zinterpretuj wyniki - które cechy są najważniejsze dla predykcji?
- Czy wyniki są zgodne z twoimi obserwacjami z EDA?

In [ ]:
# Twój kod tutaj:
# feature_importance = xgb_base.feature_importances_
# feature_names = X.columns  # jeśli X to DataFrame

# Stwórz DataFrame z ważnością cech
# importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': feature_importance
# }).sort_values('importance', ascending=False)

# Wykres 15 najważniejszych cech
# plt.figure(figsize=(10, 8))
# sns.barplot(data=importance_df.head(15), x='importance', y='feature')
# plt.title('XGBoost - 15 najważniejszych cech')
# plt.show()

### 5.3 Optymalizacja hiperparametrów XGBoost
- Użyj GridSearchCV do optymalizacji kluczowych parametrów:
  - n_estimators: [100, 200, 300]
  - max_depth: [3, 4, 5, 6]
  - learning_rate: [0.01, 0.1, 0.2]
  - subsample: [0.8, 0.9, 1.0]
- **Uwaga**: To może potrwać długo, możesz zmniejszyć zakres

In [ ]:
# Twój kod tutaj:


# print("Rozpoczynam Grid Search XGBoost...")
# grid_search_xgb.fit(X_train, y_train)

### 5.4 Ocena finalnego modelu XGBoost
- Oceń najlepszy model XGBoost na zbiorze testowym
- Stwórz confusion matrix
- Porównaj metryki z modelem SVM
- Przeanalizuj feature importance dla zoptymalizowanego modelu

In [ ]:
# Twój kod tutaj:


# print("Najlepsze parametry XGBoost:", grid_search_xgb.best_params_)
# print("XGBoost Test Accuracy:", accuracy_score(y_test, y_test_pred_xgb))
# print("\nClassification Report:")
# print(classification_report(y_test, y_test_pred_xgb, target_names=['Stayed', 'Left']))

# Zadanie 6: Porównanie modeli i wnioski biznesowe 📈

## Cel
Porównać oba modele i wyciągnąć praktyczne wnioski dla działu HR.

### 6.1 Porównanie wyników
- Stwórz tabelę porównującą oba modele:
  - Accuracy na test
  - Precision dla klasy "Left"  
  - Recall dla klasy "Left"
  - F1-score dla klasy "Left"
- Który model jest lepszy i dlaczego?

In [ ]:
# Twój kod tutaj:
import time



# Stwórz tabelę porównawczą

# print(comparison)

### 6.2 Analiza błędów
- Znajdź przypadki gdzie oba modele się myliły
- Znajdź przypadki gdzie modele się różniły w predykcji
- Przeanalizuj charakterystyki błędnie sklasyfikowanych pracowników
- Czy widzisz jakieś wzorce?

In [ ]:
# Twój kod tutaj:
# Porównanie predykcji
# comparison_df = pd.DataFrame({
#     'Actual': ,
#     'SVM_Pred': ,
#     'XGB_Pred': 
# })

# Błędy obu modeli
# both_wrong = 

# Różnice między modelami
# different_pred = 

# print(f"Przypadki gdzie oba modele się myliły: ")
# print(f"Przypadki gdzie modele różnią się: ")

### 6.3 Wnioski biznesowe
Na podstawie analizy odpowiedz na pytania:

**Dla działu HR:**
- Które czynniki najsilniej wpływają na odejścia pracowników?
- Jakie działania profilaktyczne można podjąć?
- Na których grupach pracowników HR powinien się skupić?

**Dla modelowania:**
- Który model polecasz do wdrożenia i dlaczego?
- Jakie są główne zalety i wady każdego podejścia?
- Jak można poprawić wyniki modeli?

**Twoje wnioski biznesowe:**

### Kluczowe czynniki wpływające na attrition:
<!-- Wymień 5 najważniejszych czynników i jak je interpretować -->

### Rekomendacje dla HR:
<!-- Konkretne działania które może podjąć dział HR -->

### Rekomendacje techniczne:
<!-- Który model wdrożyć, jak go monitorować, jak poprawić -->

### Ograniczenia analizy:
<!-- Co mogłoby być zrobione lepiej, jakie są ograniczenia danych -->

# Zadanie 7: Zadania dodatkowe (dla chętnych) 🌟

## Cel
Pogłębić analizę i sprawdzić zaawansowane techniki.

### 7.1 Feature Engineering
- Stwórz nowe cechy:
  - Ratio: Monthly Income / Years at Company (wzrost zarobków)
  - Age groups (młodzi/średni/starsi)
  - Interakcje między ważnymi cechami
- Sprawdź czy poprawiają one wyniki modeli

In [ ]:
# Twój kod tutaj (opcjonalnie):
# Feature engineering
# ...

### 7.2 Ensemble model
- Stwórz VotingClassifier łączący SVM i XGBoost
- Sprawdź czy ensemble daje lepsze wyniki niż pojedyncze modele
- Przetestuj różne wagi dla modeli

In [ ]:
# Twój kod tutaj (opcjonalnie):
# from sklearn.ensemble import VotingClassifier
# ensemble = VotingClassifier(...)
# ...

### 7.3 Krzywa ROC i AUC
- Narysuj krzywe ROC dla obu modeli
- Oblicz AUC score
- Zinterpretuj wyniki

In [ ]:
# Twój kod tutaj (opcjonalnie):
# from sklearn.metrics import roc_curve, auc
# ...

### 7.4 Analiza SHAP 
- Użyj biblioteki SHAP do interpretacji modelu XGBoost
- Stwórz wykresy SHAP values dla najważniejszych cech
- Przeanalizuj jak poszczególne cechy wpływają na konkretne predykcje

In [ ]:
# Twój kod tutaj (opcjonalnie):
# import shap
# explainer = shap.Explainer(best_xgb)
# shap_values = explainer(X_test[:100])  # sample dla szybkości
# shap.plots.waterfall(shap_values[0])

# Podsumowanie i refleksje 🎯

## Co osiągnąłeś w tych ćwiczeniach:
✅ Przeprowadziłeś kompleksową analizę rzeczywistego zbioru danych HR  
✅ Zaimplementowałeś i porównałeś dwa różne algorytmy ML  
✅ Dokonałeś optymalizacji hiperparametrów  
✅ Przeanalizowałeś wyniki z perspektywy biznesowej  
✅ Wyciągnąłeś praktyczne wnioski dla organizacji  

## Kluczowe umiejętności:
- **Data Science workflow**: od surowych danych do wdrożenia
- **Model comparison**: obiektywne porównanie algorytmów
- **Business insights**: tłumaczenie wyników ML na język biznesu
- **Critical thinking**: analiza ograniczeń i możliwości poprawy

## Następne kroki:
- Przetestuj inne algorytmy (Random Forest, Logistic Regression)


**Gratulacje! Ukończyłeś kompletny projekt ML od A do Z! 🎉**